In [36]:
import random
import pandas as pd
import torch
import random
import cv2
import os
import os.path as osp

import numpy as np

from pretrainedmodels.models import resnet34
import collections
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from albumentations import Resize, RandomGamma, Rotate, HorizontalFlip, Compose

In [34]:
def to_tensor(img):
    img = img.astype(np.float32) / 255.0 - 0.5
    img = img[:, :, (2, 1, 0)]  # BGR -> RGB
    return torch.from_numpy(img).permute(2, 0, 1)
        

In [86]:
class SeameseFishDs:
    def __init__(self, df, path, aug):
        self.df = df
        self.path = path
        self.aug = aug
        
    def __len__(self):
        return self.df.shape[0]
    
    def choise_same_pair(self, idx):
        new_item = random.randrange(len(self.df.Id))
        while idx != self.df.Id[new_item]:
            new_item = random.randrange(len(self.df.Id))
        
        return new_item
    
    def choose_another_pair(self, idx):
        new_item = random.randrange(len(self.df.Id))
        while idx == self.df.Id[new_item]:
            new_item = random.randrange(len(self.df.Id))
        
        return new_item
    
    def __getitem__(self, item):
        img = cv2.imread(osp.join(self.path, self.df.Image[item]))
        
        same_pair = random.random() > 0.5
        
        if same_pair:
            pair_img_item = self.choise_same_pair(self.df.Id[item])
        else:
            pair_img_item = self.choose_another_pair(self.df.Id[item])
        
        img_pair = cv2.imread(osp.join(self.path, self.df.Image[pair_img_item]))
        
        if self.aug is not None:
            img = self.aug(image=img)['image']
            img_pair = self.aug(image=img_pair)['image']
        
        img = to_tensor(img)
        img_pair = to_tensor(img_pair)
        
        label = torch.Tensor([float(same_pair)])
        
        return (img, img_pair), label
        

In [87]:
train_aug = Compose([
    HorizontalFlip(p=0.7),
    RandomGamma(p=0.7),
    Rotate(p=0.7),
    Resize(width=224, height=224)
])

valid_aug = Compose([
    Resize(width=224, height=224)
])


In [88]:
train = pd.read_csv('train.csv')
encoder = LabelEncoder()
train.Id = encoder.fit_transform(train.Id)
train, valid = train_test_split(train)
train.reset_index(inplace=True)
valid.reset_index(inplace=True)

In [89]:
train_ds = SeameseFishDs(train, 'train', train_aug)
valid_ds = SeameseFishDs(valid, 'train', valid_aug)

In [90]:
train_loader = DataLoader(train_ds, shuffle=True, num_workers=10, pin_memory=True, batch_size=32)
valid_loader = DataLoader(valid_ds, shuffle=False, num_workers=10, pin_memory=True, batch_size=32)

In [91]:
loaders = collections.OrderedDict()


In [92]:
loaders["train"] = train_loader
loaders["valid"] = valid_loader

In [93]:
model = resnet34(pretrained='imagenet')
model.last_linear = nn.Linear(512, 128)

In [94]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((label) * torch.pow(euclidean_distance, 2) + (1-label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [95]:

criterion = ContrastiveLoss()
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, weight_decay=5e-4, lr=3e-2)
# scheduler = None  # for OneCycle usage
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 8], gamma=0.3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, verbose=True)


In [96]:
import collections
from catalyst.dl.callbacks import (
    LossCallback, 
    Logger, TensorboardLogger,
    OptimizerCallback, SchedulerCallback, CheckpointCallback, 
    PrecisionCallback, OneCycleLR)

# the only tricky part
n_epochs = 10
logdir = "/tmp/runs/"

callbacks = collections.OrderedDict()

callbacks["loss"] = LossCallback()
callbacks["optimizer"] = OptimizerCallback()
callbacks["precision"] = PrecisionCallback(
    precision_args=[1, 3, 5])

# OneCylce custom scheduler callback
# callbacks["scheduler"] = OneCycleLR(
#     cycle_len=n_epochs,
#     div=3, cut_div=4, momentum_range=(0.95, 0.85))

# Pytorch scheduler callback
callbacks["scheduler"] = SchedulerCallback(
    reduce_metric="precision01")

callbacks["saver"] = CheckpointCallback()
callbacks["logger"] = Logger()
callbacks["tflogger"] = TensorboardLogger()

In [97]:
from catalyst.dl.runner import ClassificationRunner

runner = ClassificationRunner(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler)
runner.train(
    loaders=loaders, 
    callbacks=callbacks, 
    logdir=logdir,
    epochs=n_epochs, verbose=True)





0 * Epoch (train):   0% 0/595 [00:00<?, ?it/s]

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not list